# On matrix factorizations

Recall that linear operators, and more generally, linear maps can be uniquely associated with their matrices, once a particular basis is fixed.  In fact, many linear algebra textbooks take an entirely matrix-based world view, and with a good reason.  Indeed, matrices provide a simple and systematic way of representing operators in computations, which is appreciated by mathematicians, numerical software developers, and users of linear algebra - that is pretty much by everyone.
In this note we provide a very brief discussion of the most important matrix factorizations, give references to the standard routines for their computation in Python, and finally some matrix-interpretations of some of the theorems about operators we have proved in this course.

## Singular value decomposition (SVD)

Singular value decomposition states that for an arbitrary linear map $T\in\mathcal{L}(\mathcal{V},\mathcal{U})$ we can find two orthonormal bases in $\mathcal{V}$ and $\mathcal{U}$ such that the matrix of the operator with respect to these bases is diagonal with non-negative entries.  In matrix-language, we have a representation $A=U\Sigma V^*$, where If $V^*=\bar{V}^T$, that is, conjugated and transposed.  In this representation, $\Sigma$ is a diagonal matrix with singular values on the main diagonal, and $U$ and $V$ are the matrices with orthonormal columns (i.e., $U^* U = I$ and $V^* V = I$) giving us left and right singular vectors.
Let us consider an example:

In [5]:
import numpy as np
A = np.array([[1., 2., 3.],[4., 5., 6.]])
A

array([[1., 2., 3.],
       [4., 5., 6.]])

Let us now compute SVD of A

In [6]:
u,sigma,vh=np.linalg.svd(A)
u,sigma,vh

(array([[-0.3863177 , -0.92236578],
        [-0.92236578,  0.3863177 ]]),
 array([9.508032  , 0.77286964]),
 array([[-0.42866713, -0.56630692, -0.7039467 ],
        [ 0.80596391,  0.11238241, -0.58119908],
        [ 0.40824829, -0.81649658,  0.40824829]]))

Note that $\sigma$ is returned as just a list of singular values.  Let us check the orthonormality of vectors in $u$ and $v$ (note: the routine returns $v^*$ and not $v$):

In [7]:
np.conjugate(u.T) @ u, vh @ np.conjugate(vh.T)

(array([[1.00000000e+00, 5.33877946e-19],
        [5.33877946e-19, 1.00000000e+00]]),
 array([[1.00000000e+00, 5.55111512e-17, 5.55111512e-17],
        [5.55111512e-17, 1.00000000e+00, 3.88578059e-16],
        [5.55111512e-17, 3.88578059e-16, 1.00000000e+00]]))

Note the appearence of small numbers in off-diagonal entries, reminding us that we do not compute in exact arithmetics.
Let us now compute $U\Sigma V^*$:

In [8]:
u @ np.diag(sigma) @ vh[:2]

array([[1., 2., 3.],
       [4., 5., 6.]])

Note we get back $A$, however we need to select only two (first) singular vectors from $v$ out of the three basis vectors there (one can simply check the dimensions of the matrices to see this).  To avoid this, one can instead compute a "compact" version of SVD:

In [9]:
u1,sigma1,vh1=np.linalg.svd(A, full_matrices=False)
u1,sigma1,vh1

(array([[-0.3863177 , -0.92236578],
        [-0.92236578,  0.3863177 ]]),
 array([9.508032  , 0.77286964]),
 array([[-0.42866713, -0.56630692, -0.7039467 ],
        [ 0.80596391,  0.11238241, -0.58119908]]))

In [10]:
u1 @ np.diag(sigma1) @ vh1

array([[1., 2., 3.],
       [4., 5., 6.]])

SVD is expensive to compute, but is used in many contexts: least square and inverse problems (pseudoinverse), computing approximations to operators, statistics (principal component analysis), computing ranks of matrices, bases of range and null-spaces, etc.

## QR factorization

Recall that Gram-Schimdt process can be used to transform any list $(a_1,\dots,a_m)$ of linearly independent vectors into an orthonormal list $(q_1,\dots,q_m)$ with the property that $\text{span}(v_1,\dots,v_j)=\text{span}(q_1,\dots,q_j)$, $1\leq j \leq m$.
In matrix language, any matrix $A$ containing the column vectors $(a_1,\dots,a_m)$ can be expressed as $A=QR$, where the columns of $Q$ are orthonormal (i.e., $Q^* Q = I$), which corresponds to the orthonormality of the basis  $(q_1,\dots,q_m)$ encoded in the columns of $Q$, and $R$ is upper triangular, which represents the the equality between spans of $a$'s and $q$'s.
Note: $QR$ factorization is normally not implemented using the textbook Gramm-Schmidt process, which does not perform very well in inexact arithmetics.

In [11]:
import numpy as np
A = np.array([[1.,2.],[3.,4.],[5.,6.]])
A

array([[1., 2.],
       [3., 4.],
       [5., 6.]])

In [12]:
Q,R=np.linalg.qr(A)
Q,R

(array([[-0.16903085,  0.89708523],
        [-0.50709255,  0.27602622],
        [-0.84515425, -0.34503278]]),
 array([[-5.91607978, -7.43735744],
        [ 0.        ,  0.82807867]]))

Let us verufy that $Q^*Q = I$ (in the absense of round off errors) and $QR = A$:

In [13]:
np.conjugate(Q.T)@Q, Q@R

(array([[1.0000000e+00, 1.0875945e-16],
        [1.0875945e-16, 1.0000000e+00]]),
 array([[1., 2.],
        [3., 4.],
        [5., 6.]]))

QR factorization is primarily used for solving least-squares problems.  Another application of QR factorization is in eigenvalue (and singular value) computation algorithms, see below.

## Cholesky factorization

Cholesky factorization is a representation of Hermitian positive definite matrices (which correspond to positive and invertible operators) as $A=R^*R$, where $R$ is an upper triangular matrix.  Cholesky factorization is primarily utilized for solving systems of linear algebraic equations.  After the expensive part (factorization) is computed, the solution to the original linear system $Ax=b$ is computed by solving two (simple) systems with triangular matrices (so called backward-forward substitution): $R^*y = b$ and subsequently $Rx = y$.
Note: the routine below returns a lower triangular matrix $L=R^*$.

In [14]:
import numpy as np
# let us generate a random positive definite matrix
n = 5
Atmp = np.random.rand(n,n)
A = np.conjugate(Atmp.T)@Atmp + 1.0E-10*np.eye(n)
# we now compute a Cholesky factorization of A
L = np.linalg.cholesky(A)
A, L

(array([[2.53703353, 1.51194951, 1.55626314, 1.96756012, 2.43318558],
        [1.51194951, 1.38006151, 0.90802796, 1.25098167, 1.35826107],
        [1.55626314, 0.90802796, 1.7131923 , 1.18795462, 1.66792259],
        [1.96756012, 1.25098167, 1.18795462, 1.82614035, 1.86078965],
        [2.43318558, 1.35826107, 1.66792259, 1.86078965, 2.77504445]]),
 array([[ 1.59280681,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.94923596,  0.69210736,  0.        ,  0.        ,  0.        ],
        [ 0.97705706, -0.02807331,  0.87049623,  0.        ,  0.        ],
        [ 1.23527857,  0.11329286, -0.01815346,  0.53578203,  0.        ],
        [ 1.52760873, -0.13263848,  0.19717271, -0.01423398,  0.6203091 ]]))

Let us not compute the norm of the difference between $A$ and $LL^*$:

In [15]:
np.linalg.norm(A-L@np.conjugate(L.T))

7.364386412590295e-16

which is a very small number, same order of magnitude as the round off error.

In the context of solving linear systems, we proceed as follows.

In [16]:
b = np.random.rand(n)
y = np.linalg.solve(L,b)
x = np.linalg.solve(np.conjugate(L).T,y)
np.linalg.norm( x - np.linalg.solve(A,b) )

1.6504651808933464e-15

So indeed, the difference between the solutions to $Ax=b$ and those to $Ly=b$, $L^*x = y$ is very small.

## LU factorization

LU-factorization is a representation of a square matrix as a product of two triangular matrices: a lower triangular matrix $L$, and an upper triangular matrix $U$.  LU factorization is equivalent to a matrix-representation of Gaussian elimination.  Its computation costs approximately twice as much as the computation of Cholesky factorization, but the factorized matrix does not have to be symmetric (real case)/Hermitian (complex case), or positive definite.  Assuming that the representation $A=LU$ exists, one can compute the elements of $L$ and $U$ one by one, starting from the top left corner of $A$.  There is ambiguity in the computation of diagonal elements of $L$ and $U$, which is typically removed by assuming that $L_{ii}=1$.
Unfortunately, not all invertible matrices admit an $LU$ factorization.  To deal with this issue, we allow the rows of the matrix $A$ to be permuted, which results in a factorization $A=PLU$, where $P$ is just a permutation matrix.
Such decompositions exist for arbitrary invertible matrices, and are called LU factoriszation with pivoting.  Once the factorization is computed, one solves the system $Ax=b$ as $Ly = P^T b$, $Ux = y$, similarly to the case of Cholesky:

In [17]:
import numpy as np
import scipy.linalg as sl
n = 5
A = np.tril(np.reshape(np.arange(n*n)+1,(n,n)))
P,L,U = sl.lu(A)
np.linalg.norm(A-P@L@U)

1.7763568394002505e-15

Let us now solve a linear system and check the result:

In [18]:
b = np.random.rand(n)
y = np.linalg.solve(L,P.T@b)
x = np.linalg.solve(U,y)
np.linalg.norm(x - np.linalg.solve(A,b))

0.0

## Eigenvalue decomposition

Recall that normal operators (complex case) and aelf-adjoint operators (real case) can be diagonalized using an orthonormal basis of eigenvectors.  In the matrix language, for example in the real case we can represent any $A$ as a product $VDV^T$ with $D$ being a diagonal matrix containing the eigenvalues on the diagonal, and columns $V$ representing the orthonormal eigevectors, i.e., $V^*V = I$.
For example:

In [19]:
import numpy as np
import scipy.linalg as sl
n = 5
Atmp = np.random.rand(n,n)
A = Atmp.T @ Atmp
d,V = sl.eigh(A)
d,V

(array([3.58447642e-04, 4.74927664e-02, 3.26943232e-01, 7.39921632e-01,
        9.99674337e+00]),
 array([[ 0.47226796, -0.64334721,  0.19717323,  0.37628869, -0.42731357],
        [-0.61654676,  0.1528263 ,  0.58255302,  0.2013031 , -0.46542803],
        [-0.36944232, -0.2334876 , -0.69716683, -0.18169019, -0.53846355],
        [ 0.35087338,  0.68679412, -0.25574958,  0.45715356, -0.36166902],
        [ 0.37045015,  0.19116974,  0.26515137, -0.75886967, -0.42430225]]))

In [20]:
np.linalg.norm(V @ np.diag(d) @ V.T - A), np.linalg.norm(V.T@V - np.eye(n))

(1.9856549075576475e-14, 7.322422964396005e-16)

Of course one can also request to compute the eigenvalues/vectors of non-normal operators, however in this case we cannot expect the eigenvectors to produce a basis of the space (or to be orthonormal):

In [21]:
import numpy as np
import scipy.linalg as sl
n = 3
N = np.diag(np.ones(n),1)
N

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [22]:
d,V = sl.eig(N)
d, V

(array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([[ 1.00000000e+000, -1.00000000e+000,  1.00000000e+000,
         -1.00000000e+000],
        [ 0.00000000e+000,  4.00833672e-292, -4.00833672e-292,
          4.00833672e-292],
        [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         -0.00000000e+000],
        [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
          0.00000000e+000]]))

As expected, this nilpotent matrix has only one eigenvalue zero of multiplicity equal to the size of the matrix. There is only one (non-generalized) eigenvector, and columns of $V$ are all the same (hence also lineraly dependent) and do not constitute the basis.

## Jordan form

Jordan form computation is numerically unstable, because it relies on identifying whether the computed eigenvalues are exactly identical (rarely happens in floating point arithmetics) or not.  For small matrices, one can perform a symbolic computation:

In [23]:
from sympy.matrices import Matrix
A = Matrix([[ 6,  5, -2, -3], [-3, -1,  3,  3], [ 2,  1, -2, -3], [-1,  1,  5,  5]])
P, J = A.jordan_form()
J

Matrix([
[2, 1, 0, 0],
[0, 2, 0, 0],
[0, 0, 2, 1],
[0, 0, 0, 2]])

Let us now try the same using floating point arithmetics:

In [24]:
import numpy as np
import scipy.linalg as sl
A_numpy = np.array(A,dtype=np.float64)
d,V = sl.eig(A_numpy)
d, V

(array([2.00000005+0.j, 1.99999995+0.j, 1.99999996+0.j, 2.00000004+0.j]),
 array([[ 0.73029675, -0.73029674,  0.60135886, -0.60135888],
        [-0.54772255,  0.54772256, -0.52944886,  0.52944886],
        [ 0.36514837, -0.36514837,  0.45753886, -0.45753885],
        [-0.18257418,  0.18257419, -0.38562885,  0.38562884]]))

One can see that all the computed eigenvalues are (slightly) different, and the corresponding eigenvectors are therefore linearly independent (or only almost linearly dependent).

## Schur decomposition

Remember that in a complex vector space for an arbitrary operator there is a basis, in which the operator's matrix is upper triangular.  By applying Gram-Schmidt orthogonalization to such a basis, we conclude that the same is true with an additional assertion that the basis can be selected to be orthonormal.  This theorem is due to Schur, and the corresponding matrix factorization $A = Q R Q^*$ is known as Schur factorization:

In [27]:
import numpy as np
import scipy.linalg as sl
n = 10
A = np.random.rand(n,n)
R, Q = sl.schur(A,output='complex')
R, Q, np.linalg.norm(np.conjugate(Q.T)@Q-np.eye(n))

(array([[ 5.22017787e+00-5.13568015e-16j, -2.33374845e-01+1.93320337e-01j,
         -5.94006001e-01-1.51015397e-01j,  3.71176558e-01+7.68488118e-02j,
         -3.34602935e-02+8.28947292e-02j, -7.24324475e-03-6.43658449e-02j,
         -1.45030762e-01+4.13250948e-01j,  3.91048974e-01-1.17596487e-01j,
          6.41138617e-02+3.98298525e-02j,  1.53182670e-01+3.13310727e-02j],
        [ 0.00000000e+00+0.00000000e+00j, -1.32147170e+00+1.39676539e-17j,
          1.49723289e-01+9.56416130e-02j,  2.13110954e-02-1.40626769e-01j,
         -6.32784780e-02+1.14879246e-01j, -9.47718921e-03+1.93328518e-01j,
         -1.19013611e-01+4.41356561e-02j,  1.12456196e-01+2.01226885e-02j,
         -1.80331511e-01-5.38558034e-01j,  4.25303776e-02+5.28910330e-02j],
        [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
          5.48615829e-01+5.75613290e-01j, -2.64044138e-02+5.76510684e-01j,
          2.82320868e-01+6.38298357e-02j,  6.36650166e-02+1.02491323e-01j,
         -2.79920022e-0

Recall that $A$ and $R$ are matrices correspoinding to the same operator, just in different bases. The eigenvalues of a triangular matrix are on the diagonal, so computation of the Schur form is equivalent to computing all the eigenvalues of the matrix (and is therefore quite expensive).  In fact, one of the popular eigenvalue computation algorithms, QR-algorithm, iteratively transforms a matrix into a Schur form.

## QR algorithm for computing eigenvalues

We have previously considered power iteration algorithm for computing the eigenvalue of a matrix with the largest magnitude.  One can extend this idea for computing all eigenvalues simultaneously by considering the following iteration: $A_0=A$, $Q_k R_k = A_k$, $A_{k+1} = R_k Q_k$, $k=0,1,\dots$
One can expect, under certain conditions, that $A_k$ converges to an upper triangular matrix. In fact, 
$A^{k+1} = Q_0 R_0 Q_0 R_0 \dots Q_0 R_0 = Q_0 A_1 \dots A_1 R_0 = Q_0 Q_1 R_1 \dots Q_1 R_1 R_0 = \dots$.  Therefore, $Q = Q_0 Q_1 \dots Q_k$ and $R = R_k R_{k-1} \dots R_0$ constitute the QR factorization of the "power iteration" matrix $A^{k+1}$.
Omitting several important details, the simplest implenentation of such an eigenvalue computation algorithm is shown below:

In [30]:
import numpy as np
import scipy.linalg as sl
n = 5
Atmp = np.random.rand(n,n)
A = Atmp.T @ Atmp
d,V = sl.eigh(A)
print("Eigenvalues of A, scipy:")
print(d)

epsilon = 1.0E-08
while np.linalg.norm(np.tril(A,-1)) > epsilon*np.linalg.norm(A):
    Q,R = np.linalg.qr(A)
    A   = R@Q
print("Approximate eigenvalues of A, QR-algorithm")
print(np.sort(np.diag(A)))


Eigenvalues of A, scipy:
[1.86730047e-07 8.05589058e-02 3.32879116e-01 1.49961724e+00
 6.49927971e+00]
Approximate eigenvalues of A, QR-algorithm
[1.86730047e-07 8.05589058e-02 3.32879116e-01 1.49961724e+00
 6.49927971e+00]


Since singular values of an operator are simply the eigenvalues of a related self-adjoint operator, the same algorithm can be used for computing singular values as well.

## Some concluding remarks

For all but the tiniest matrices $A$, avoid computing $\det(A)$ and $A^{-1}$, as these computations are both expensive and get inaccurate very fast because of round-off errors. Instead of computing $A^{-1}$, factorize the matrix: use Cholesky, whenever possible (for positive definite Hermitian matrices), LU with pivoting (permutations), when not.  These factorizations allow one to quickly solve systems of linear algebraic equations, that is,  apply the operator associated with the matrix $A^{-1}$.  For particularly difficult systems of linear algebraic equations, or least squares problems, QR factorization could be more appropriate.  Computing all singular or eigenvalues is very expensive, and is practically impossible for all but relatively small problems.  In fact, even computing Cholesky, LU, or QR factorization becomes prohibitevly expensive for large problems - and special iterative algorithms are instead utilized for solving systems of linear algebraic equations.